In [1]:
from fyenn_class import  pd_loaddata, pd, np 
from datetime import datetime, timedelta

path = '../data_down/'
os.listdir(path)


['huawei_daily.csv', 'huawei_output.csv', '入湖数据关系.csv', '入湖数据关系.xlsx']

In [7]:
'{note:=>50}'.format(note='Lorem') + '{note:=>50}'.format(note='')
# rel 

'=============================================Lorem=================================================='

In [8]:
%%time
huawei_output = pd_loaddata.pd_csv(path + 'huawei_output.csv')
# huawei_daily = pd_loaddata.pd_csv(path + 'huawei_daily.csv')
# rel = pd_loaddata.pd_excel(path + '入湖数据关系', 0)
# huawei_output.shape
# pd_loaddata.pd_show(15, None)
# rel = rel.drop(['部门', '站点'],axis = 1).drop_duplicates();rel
def datetime_(coach, col):
    """
    detail time to date; 
    split and assign to 3 cols of year m d
    """
    coach[col] = coach[col].astype(str).str.slice(0,10)
    coach = pd.concat([coach, pd.DataFrame(list(coach[col].str.split('-')))], axis =1)
    coach = coach[coach[[0,1,2]].astype(int).sum(axis = 1) != 0]
    coach = coach.rename({0:'year', 1:'month', 2: 'date'}, axis=1)
    return coach
huawei_output = datetime_(coach=huawei_output, col='update_date').drop(['createtime', 'updatetime'], axis = 1)
huawei_output = huawei_output.drop(['id','appid','entryid'], axis = 1).drop_duplicates().sort_values('update_date')
def search_col(df, str):
    """
    列名正则搜索.
    """
    return list(pd.Series(df.columns)[pd.Series(df.columns).str.match(str)])
# 识别异常列名
tran_col = list(pd.Series(huawei_output.columns)[
    pd.Series(huawei_output.columns).str.contains(
        '(receive|psn|send|transport_times|update_date|year|month|date|id)'
        ) == False
        ])


# clean , case, withdraw)

Wall time: 467 ms


<timed exec>:26: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [17]:
pd.Series(huawei_output.columns)[pd.Series(huawei_output.columns).str.contains('te\_')]

0            update_date
21     te_origin_receive
22        te_origin_send
23    te_product_receive
24       te_product_send
dtype: object

In [18]:
huawei_output['te_origin_receive'] = huawei_output['te_origin_receive'] + huawei_output['te_product_receive'] 
huawei_output['te_origin_send'] = huawei_output['te_origin_send'] + huawei_output['te_product_send'] 
huawei_output = huawei_output.drop(['te_product_send', 'te_product_receive'], axis = 1)

In [22]:

"""
ou 和正则匹配
"""
# relist = ['hon', 'origi', 'pearl', 'guiy', '^t\_', 'r4\_', 'nanh', 'ansh', 'te']
relist = ['origi', 'hon', 'pearl', 'guiy', '^t\_', 'r4\_', 'nanh', 'ansh', 'te\_']
oulist = ['HUAWEDHW4S',
    'HONORDGHMS',
    'HUAWEDHWTS',
    'HUAWEDGTRD',
    'HUAWEDGLSS',
    'HUAWEDHW1S',
    'HUAWEDGNHS',
    'NEXPEDGWHS',
    'TYCOTSDXXS',]
    
del dict 
dict = dict(zip(relist, oulist))
# re = 'pear'
# pd.Series(huawei_output.columns).str.extract("(" + re + "[a-z]+)").dropna()
dict

{'origi': 'HUAWEDHW4S',
 'hon': 'HONORDGHMS',
 'pearl': 'HUAWEDHWTS',
 'guiy': 'HUAWEDGTRD',
 '^t\\_': 'HUAWEDGLSS',
 'r4\\_': 'HUAWEDHW1S',
 'nanh': 'HUAWEDGNHS',
 'ansh': 'NEXPEDGWHS',
 'te\\_': 'TYCOTSDXXS'}

In [23]:
def concat_(re, ou):
    huawei_output[search_col(huawei_output, re)].shape
    # n = 4 - huawei_output[search_col(huawei_output, re)].shape[1]
    # m = huawei_output[search_col(huawei_output, re)].shape[0]
    # print(m, n)
    data = pd.concat(
        [
            huawei_output[search_col(huawei_output, re)], 
            # pd.DataFrame(np.zeros(shape=(m, n), dtype=int)),
            huawei_output[['update_date', 'year', 'month', 'date']]
        ]
        , axis = 1).sort_values('update_date')
    # print(data.shape)
    data.columns = list(pd.Series(data.columns).str.extract(
        '(receive|psn|send|transport_times|update_date|year|month|date)'
        )[0])
    try:
        data = data.rename({np.nan:'addition'}, axis = 1)
    except:
        pass
    data['ou'] = ou
    data['station'] = pd.Series(huawei_output.columns).str.extract("(" + re + "[a-z]+)").dropna().iloc[0,0]
    data['addition_type'] = pd.Series(tran_col).str.extract("(" + re + ".+)").fillna('z').sort_values(0).iloc[0,0]
     
    return data.reset_index(drop=True)

df = pd.DataFrame()
for re in dict:
    print(re, dict[re])
    df = pd.concat([df, concat_(re, dict[re])], axis = 0)


origi HUAWEDHW4S
hon HONORDGHMS
pearl HUAWEDHWTS
guiy HUAWEDGTRD
^t\_ HUAWEDGLSS
r4\_ HUAWEDHW1S
nanh HUAWEDGNHS
ansh NEXPEDGWHS
te\_ TYCOTSDXXS


In [24]:
pd.Series(tran_col).str.extract("(" + 'pear' + ".+)").fillna('zull')
# pd.Series(huawei_output.columns).str.extract("(" + 't_' + ".+)").dropna().iloc[0,0]
pd.Series(tran_col).str.extract("(" + 'anshi' + ".+)").fillna('z').sort_values(0).iloc[0,0]

'anshi_case'

In [25]:
def cleanm(df):
    """
    清楚每日填报重复, 时间日期drop, update_date 日期加一以匹配前端.
    """
    df = df.fillna(0).drop_duplicates()
    df['flag_sum'] = df[['receive', 'send', 'psn']].sum(axis =1)
    df = df.sort_values(['update_date', 'ou', 'flag_sum'], ascending=False).groupby(
        [
            'update_date', 'ou'
        ]
        ).first().reset_index()
    try:
        df = df.drop(['year', 'month', 'date', 'flag_sum'], axis = 1)
    except:
        pass

    df['update_date'] = pd.to_datetime(df['update_date']) + timedelta(days = 1)
    df['addition_type'] = df['addition_type'].str.replace('^z', 'None')
    return df

df = cleanm(df)

C:\Users\DSCSHA~1\AppData\Local\Temp/ipykernel_10992/680750927.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  df['addition_type'] = df['addition_type'].str.replace('^z', 'None')


In [26]:
huawei_output[huawei_output['update_date'] == '2021-11-27']

,update_date,origin_receive,origin_send,origin_psn,honor_receive,honer_send,honor_psn,pearlriver_transport_times,guiyang_transport_times,t_product_receive,...,anshi_send,anshi_case,te_origin_receive,te_origin_send,pre_receive,pre_withdraw,honor_transport_times,year,month,date
11630,2021-11-27,781.0,1572.0,7610.0,302.0,528.0,3340.0,0.0,0.0,NaN,...,33292.0,0.0,5885.0,1247.0,0.0,0.0,1.0,2021,11,27


In [185]:
df[['receive', 'send', 'psn', 'transport_times', 'addition']] = df[
        ['receive', 'send', 'psn', 'transport_times', 'addition']
        ].astype(int)

In [186]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2656 entries, 0 to 2655
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   update_date      2656 non-null   datetime64[ns]
 1   ou               2656 non-null   object        
 2   receive          2656 non-null   int32         
 3   send             2656 non-null   int32         
 4   psn              2656 non-null   int32         
 5   transport_times  2656 non-null   int32         
 6   station          2656 non-null   object        
 7   addition_type    2656 non-null   object        
 8   addition         2656 non-null   int32         
dtypes: datetime64[ns](1), int32(5), object(3)
memory usage: 135.0+ KB


In [180]:
df.query("update_date == '2021-11-27'")

,update_date,ou,receive,send,psn,transport_times,station,addition_type,addition
2640,2021-11-27,HONORDGHMS,2111.0,4245.0,7340.0,0.0,origin,None,0.0
2641,2021-11-27,HUAWEDGLSS,0.0,0.0,0.0,0.0,t_product,t_product_clean,0.0
2642,2021-11-27,HUAWEDGNHS,271.0,1740.0,4.0,0.0,nanhua,None,0.0
2643,2021-11-27,HUAWEDGTRD,0.0,0.0,0.0,1.0,guiyang,None,0.0
2644,2021-11-27,HUAWEDHW1S,37.0,30.0,3.0,0.0,r4_receive,None,0.0
2645,2021-11-27,HUAWEDHW4S,929.0,3004.0,11169.0,0.0,honor,None,0.0
2646,2021-11-27,HUAWEDHWTS,0.0,0.0,0.0,5.0,pearlriver,None,0.0
2647,2021-11-27,NEXPEDGIHS,50154.0,26431.0,0.0,0.0,anshi,anshi_case,2086.0


In [ ]:
list(pd.Series(huawei_output.columns).str.extract("(" + 'pearl' + "[a-z]+)").dropna()[0])[0]

In [146]:
print('duplicates check',df[df[['update_date', 'ou']].duplicated()].sort_values(['update_date', 'ou']))

duplicates check Empty DataFrame
Columns: [update_date, ou, receive, send, psn, transport_times, station, addition_type, addition]
Index: []
